In [2]:
import json
import torch

with open('model_info.json', 'r') as f:
    model_info = json.load(f)

with open('../../BarTracking/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=4)
    
for model_path in model_info.keys():
    model = torch.load(model_path, map_location='cpu')
    torch.onnx.export(model, torch.rand(1, 3, 320, 320), '../../BarTracking/' + model_path.replace('.pth', '.onnx'), export_params=True, opset_version=11)

/root/workspace/bar_tracking/venv/lib/python3.9/site-packages/torch/nn/functional.py:2498: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  _verify_batch_size([input.size(0) * input.size(1) // num_groups, num_groups] + list(input.size()[2:]))
/root/workspace/bar_tracking/venv/lib/python3.9/site-packages/timm/models/layers/padding.py:19: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return max((math.ceil(x / s) - 1) * s + (k - 1) * d + 1 - x, 0)
/ro

In [4]:
import os
import onnx
import onnxsim

# list all files that are onnx
models = os.listdir('../../BarTracking/')
models = [model for model in models if model.endswith('.onnx')]
for model in models:
    # load the model
    onnx_model = onnx.load('../../BarTracking/' + model)
    # run the model
    optim_model, success = onnxsim.simplify(onnx_model)
    assert success
    onnx.save(optim_model, '../../BarTracking/' + model.replace('.onnx', '_simplified.onnx'))
    


In [1]:
import onnxruntime as ort
import numpy as np

def load_onnx_model(onnx_model_file):
   try:
       session = ort.InferenceSession(onnx_model_file, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

   except (TypeError, RuntimeError) as e:
       # It is possible for there to be a mismatch between the onnxruntime and the
       # version of the onnx model format.
       print(e)
       raise e
   return session

def onnx_infer(onnx_session, image_samples):
   input_name = onnx_session.get_inputs()[0].name
   result = onnx_session.run(None, {input_name: image_samples})
   return result[0]

In [ ]:
ses = load_onnx_model('./timm-mobilenetv3_small_minimal_100_model.onnx')